# Progamming Assignment 2
## CS 776 Evolutionary Computing: Fall 2020
## David Gabriel

#### (10 points) Given a one hill problem over 150 bits, what is the time complexity of your algorithm to find the optimum in terms of the number of evaluations? 

#### (10 points) Given an unknown problem over 150 bits, what is the time complexity of your algorithm to find the optimum in terms of the number of evaluations? 

####  (10 points) What is the space complexity? 

#### Is the algorithm complete? 

#### Is the algorithm guaranteed to find the optimum? 

#### If the algorithm finds the optimum, will it find it in minimum time? 


#### If the algorithm finds the optimum, will it find it in minimum time? 

#### 1  (20 points) What is the time complexity of your algorithm? In terms of number of evaluations. If you cannot compute time complexity in general, answer a simpler version of the question: 
For the simple one hill problem, the time complexity, as measured by number of evalution calls was measured with the following method.

For 100 solution attempts with initial conditions of unbiased random bit vectors, the total number of evalautions was 38073, summing across all attempts. All attempts were successful. That is 380.73 evaluation calls per solution. I have run other trials attempting to improve this, but this was the best version.
Because the initial conditions are uniformly distributed without bias, the expected value of the Hamming distance between the initial and solution vectors is the average number of moves required. In this case, E|H(Initial,Final)| = 50. Therefor, the algorithm solves on average 50 necessary moves with 380.73 evaluation calls. 

This is a reasonable and practical solution efficiency, but is sub optimal. The expected value of the Hamming distance is a good metric against which to judge, as is the state vector size of 100. Single hill variant ought to be solvable with an average number of solution attempts somewhere closer to the state vector length. 

#### 2 (10 points) What is the space complexity? 

From a guppy report of memory usage:

Partition of a set of 282117 objects. Total size = 33617898 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)

The algorithm uses 33617898 bytes of memory. This is a rather constant amount of memory used between iterations. I observed less than 1 kB variation betweeen iterations.
 

#### 3 Is the algorithm complete? 

Yes, the algorithm is complete, in the sense that it will find any solution of the class of either problem. If the bit order were to be randomized, the algorithm would still successfully find the optimum. If the number of bits in the information sparse sector of the 100 bit code were changed, the algorithm would auto calculate these bits and locations arbitrarily distributed throughout the bit string, adjusting the search space. All locations of known bits and information sparse bits are determined on the fly. The algorithm can optomize problems of this class well, but is not time efficient for other classes of state space with varying degrees and topologies of fitness information. 

#### Is the algorithm guaranteed to find the optimum?
Yes. Even if it gets stuck, it will enumerate every remaining untested solution, maximizing information usage, and iterate through the entire state space as necessary to determine a solution. Potential errata behavior if state space exceeds ordinary local file system and memory limitations. 

The search space is very large for the part 2 question. System hardware limitations may be a factor.

#### If the algorithm finds the optimum, will it find it minimum time? 
Well, a hash map would be faster. Or an Oracle machine.

Low hanging fruit to improve this algorithm close to idea would be to not evaluate previously changed bits, with a registry type technique for known bits similar to what I used in part 2. 

To more thoroughly answer this, I must make an assumption about "minimum time". I will assume minimum time means the average number of evaluation calls, from several trials from a random starting position, with potentially shifted or rearranged bits. This would apply for either class of problem, 'eval' or 'eval1'. 

For class of eval like problems, the information dense state space, this will find the answer in more than the minimum number of evaluation calls, compared to other algorithms that will solve this complete class of problems as defined above.

For the class of eval1 like problems, it is difficult for me to ascertain exactly what the minimum size is, for every approach. I cannot say exactly what the fitness information from eval1 is for the entire space. Therefor I cannot make concrete statements about what minimum time is. Practically speaking, the read/write locks on the files used to transfer data between python and c slow operation. Properly multi threaded optimized could would be faster.

Because of the way I coded several items to auto adjust sizes of information dense and sparse bitspace and check stalled hill climbers results in evaluations that are not absolutely necessary


# Code
## Part 2
Code is better viewed at: 
https://github.com/davidgabriel42/CS776-PA1/blob/master/python_wrapper.ipynb

In [5]:
import subprocess
import timeit
import os
import time
import math
import random
#path = '/home/dave/Documents/unr/fall20/cs776/PA1/foo'



In [2]:
class hill_climb:
    
    class climber:
        
        def __init__(self):
            
            self.combo = self.make_rand_position()
            
            self.fitness = 0
            self.n_evals = 0

        def evaluate(self, combo):

            f=open('input.txt','w+')
            for item in combo:
                f.write(str(item)+'\n')
                
            f.close()

            os.system('./cpp_eval')
            
            f = open('fitness_out.txt', 'r')
            fitness = f.read()
            f.close()
            
            self.n_evals = self.n_evals + 1
            
            #print('fit',fitness)
            return fitness

        def make_rand_position(self):
            rand_position = [0]*100
            for i in range(0,100):
                rand_position[i] = str(int(random.random() > 0.5))
            return rand_position

        def iterative_solver( self, init_position, n_iterations):
            best_known_fitness = 0
            new_position = init_position.copy()
            previous_position = init_position.copy()
            best_known_fitness = self.evaluate(init_position)
            
            for iterations in range(n_iterations):
                previous_position = new_position.copy()
                
                for key in list(range(100)):

                    candidate_position = new_position.copy()
                    candidate_position[key] = int(not int(new_position[key]))
                    
                    candidate_fitness = self.evaluate(candidate_position)

                    if int(candidate_fitness) > int(best_known_fitness):
                        print(candidate_fitness)
                        if candidate_fitness == 100:
                            print('solved')
                            return candidate_fitness

                        new_position = candidate_position.copy()
                        best_known_fitness = candidate_fitness
                        
            return best_known_fitness

In [4]:
solution = []

In [ ]:
my_climber = hill_climb.climber()
    
n_trials = 100

for i in range(n_trials):
    
    init_pos = my_climber.make_rand_position()

    solution.append(my_climber.iterative_solver(init_pos, 4))
    
    
print('Evaluations per solution = ' ,my_climber.n_evals/n_trials)

    
    

## C File
Please note this file is same for both part 1 and part 2. Linking done via CLI compilation using g++ controls object linked and therefor the black box.

In [ ]:
//main.cpp, to build use:
//g++ main.cpp eval1.o -o cpp_eval
// reading a text file
#include <iostream>
#include <fstream>
#include <string>

using namespace std;

double eval(int *pj);


int main ()
{
  int vec[100];
  int i = 0;


  ifstream File;
  File.open("input.txt");

  while(!File.eof())
  {
    File >> vec[i];
    i++;
  }

  double fitness;
  fitness = eval(vec);

  ofstream myfile_out;
  myfile_out.open ("fitness_out.txt");


  myfile_out << fitness << endl;
  myfile_out.close();
  return 0;  

}


# Code
## Part 2
Code is better viewed at: 
https://github.com/davidgabriel42/CS776-PA1/blob/master/part-2.ipynb

In [ ]:
import subprocess
import timeit
import os
import time
import math
import random
#path = '/home/dave/Documents/unr/fall20/cs776/PA1/foo'

In [6]:
class hill_climb:
    
    class climber:
        
        def __init__(self):
            
            self.combo = self.make_rand_position()
            self.combo = [0]*100
            self.fitness = 0
            self.n_evals = 0
        def show(self):
            
            print(self.combo)
        
        def evaluate(self, combo):

            f=open('input.txt','w+')
            for item in combo:
                f.write(str(item)+'\n')
                
            f.close()

            os.system('./cpp_eval')
            
            f = open('fitness_out.txt', 'r')
            fitness = f.read()
            f.close()
            
            self.n_evals = self.n_evals + 1
            
            #print('fit',fitness)
            return fitness
        
        def update(self,combo):
            fitness = self.evaluate()
            #print('fit',fitness)

            return fitness

        def make_rand_position(self):
            rand_position = [0]*100
            for i in range(0,100):
                rand_position[i] = str(int(random.random() > 0.5))
            return rand_position

        def inject_noise(self,position):
            new_position = position.copy()
            noise = self.make_rand_position()
            for i in range(0,100):
                if int(noise[i]):
                    new_position[i] = str(int(not int(new_position[i])))
            #print(noise)
            #print(position)
            return new_position
        
        def map_differentials(self,position):
            diff_list = []

            for i in range(0,100):

                row = position.copy()
                row[i] = int(not int(row[i]))
                #print('position', i, 'from' , position[i],'to',row[i] )
                diff_list.append(row)

            return diff_list

        def update_differential(self,diff_list, previous_position):

            i = 0
            starting_fitness = self.evaluate(previous_position)

            differential_all_dimensions = {}

            for row in diff_list:
                fitness = self.evaluate(row)
                differential =  int(fitness) -int(starting_fitness) 
                differential_all_dimensions.update({str(i): differential})
                i = i +1

            return differential_all_dimensions

        def iterative_solver( self, init_position, n_iterations):
            self.climber_stuck = False
            self.known_keys = []
            previous_position = init_position.copy()
            best_known_fitness = self.evaluate(init_position)
            
            for iterations in range(n_iterations):
                
                if self.climber_stuck is False:

                    new_position = previous_position.copy()

                    for key in range(100):
    
                        candidate_position = new_position.copy()
                        candidate_position[key] = int(not new_position[key])
                        candidate_fitness = self.evaluate(candidate_position)
                
                        if candidate_fitness > best_known_fitness:
                            self.known_keys.append(key)
                            
                            if candidate_fitness == 100:
                                return candidate_fitness
                            
                            new_position = candidate_position
                            best_known_fitness = candidate_fitness
                            print('fitness',best_known_fitness)
                            
                    if previous_position == new_position:
                        self.climber_stuck = True
                        
                    previous_position = new_position
                    
                else:
                    #logic for dealing with stuck hill climber
                    print('stuck at:', previous_position)
                    #enumerate rest of state space to search

                    self.unknown_keys = []

                    self.unknown_keys = list(set(range(100)) -set(self.known_keys) )
                    
                    #check all bits not checked
                    for key in self.unknown_keys:
                        candidate_position = previous_position.copy()
                        candidate_position[key] = int(not previous_position[key])
                        candidate_fitness = self.evaluate(candidate_position)
                        if candidate_fitness < best_known_fitness:
                            self.known_keys.append(key)
                            
                    self.unknown_keys = list(set(range(100)) -set(self.known_keys) )
                    
                    n_unknowns = len(self.unknown_keys)
                    
                    search_vector_max = 2**len(self.unknown_keys)
    
                    search_vector = search_vector_max/2
                    solved = False
                    
                    candidate_position = previous_position.copy()
                    print('unkown terms', len(self.unknown_keys),'max search terms',search_vector_max)
                    
                    while solved is False:
                        candidate_bits = format(search_vector, 'b')
                        #pad_zeros 
                        candidate_bits = candidate_bits.zfill(n_unknowns)
                        print(candidate_bits)
                        
                        for i in range(n_unknowns):
                            candidate_position[i] = candidate_bits[i]
                        
                        candidate_fitness = self.evaluate(candidate_position)
                        
                        if candidate_fitness > best_known_fitness:
                            previous_position = candidate_position
                        
                        search_vector = search_vector + 1
                        
                        if search_vector == search_vector_max:
                            solved = True
                            
                        if candidate_fitness == 100:
                            return candidate_fitness
                        #candidate_position = previous_position.copy()
                        
                        
                    
                    print('kk',(self.known_keys),'\nuk', (self.unknown_keys))

In [ ]:
my_climber = hill_climb.climber()
solution = []

In [ ]:
my_climber = hill_climb.climber()
    
n_trials = 1

for i in range(n_trials):
    
    init_pos = my_climber.make_rand_position()
    init_pos = [1]*100
    #print(init_pos)
    solution.append(my_climber.iterative_solver(init_pos, 4))
    #print(solution[i])

    
    

for i in range(n_trials):
    print(solution[i])


In [ ]:
"""
Given the following function:
    y = f(w1:w6) = w1x1 + w2x2 + w3x3 + w4x4 + w5x5 + 6wx6
    where (x1,x2,x3,x4,x5,x6)=(4,-2,3.5,5,-11,-4.7) and y=44
What are the best values for the 6 weights (w1 to w6)? We are going to use the genetic algorithm to optimize this function.
"""

function_inputs = [0]*150 # Function inputs.
desired_output = 44 # Function output.

def fitness_func(solution, solution_idx):
    # Calculating the fitness value of each solution in the current population.
    # The fitness function calulates the sum of products between each input and its corresponding weight.
    print(solution)
    output = numpy.sum(solution*function_inputs)
    fitness = int(my_climber.evaluate(solution))
    #fitness = 1.0 / numpy.abs(output - desired_output)
    print(fitness)
    return fitness

fitness_function = fitness_func

num_generations = 100 # Number of generations.
num_parents_mating = 7 # Number of solutions to be selected as parents in the mating pool.

# To prepare the initial population, there are 2 ways:
# 1) Prepare it yourself and pass it to the initial_population parameter. This way is useful when the user wants to start the genetic algorithm with a custom initial population.
# 2) Assign valid integer values to the sol_per_pop and num_genes parameters. If the initial_population parameter exists, then the sol_per_pop and num_genes parameters are useless.
sol_per_pop = 50 # Number of solutions in the population.
num_genes = 150

init_range_low = 0
init_range_high = 2

parent_selection_type = "sss" # Type of parent selection.
keep_parents = 7 # Number of parents to keep in the next population. -1 means keep all parents and 0 means keep nothing.

crossover_type = "single_point" # Type of the crossover operator.

# Parameters of the mutation operation.
mutation_type = "random" # Type of the mutation operator.
mutation_percent_genes = 10 # Percentage of genes to mutate. This parameter has no action if the parameter mutation_num_genes exists or when mutation_type is None.

last_fitness = 0
def callback_generation(ga_instance):
    global last_fitness
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))
    print("Change     = {change}".format(change=ga_instance.best_solution()[1] - last_fitness))
    last_fitness = ga_instance.best_solution()[1]

# Creating an instance of the GA class inside the ga module. Some parameters are initialized within the constructor.
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating, 
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop, 
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       callback_generation=callback_generation,
                       gene_type = int,
                       gene_space = [0,1])

# Running the GA to optimize the parameters of the function.
ga_instance.run()

# After the generations complete, some plots are showed that summarize the how the outputs/fitenss values evolve over generations.
ga_instance.plot_result()

# Returning the details of the best solution.
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))
print("Index of the best solution : {solution_idx}".format(solution_idx=solution_idx))

prediction = numpy.sum(numpy.array(function_inputs)*solution)
print("Predicted output based on the best solution : {prediction}".format(prediction=prediction))

if ga_instance.best_solution_generation != -1:
    print("Best fitness value reached after {best_solution_generation} generations.".format(best_solution_generation=ga_instance.best_solution_generation))

# Saving the GA instance.
filename = 'genetic' # The filename to which the instance is saved. The name is without extension.
ga_instance.save(filename=filename)

# Loading the saved GA instance.
loaded_ga_instance = pygad.load(filename=filename)
loaded_ga_instance.plot_result()
